<a href="https://colab.research.google.com/github/echukwuka/Cookbook/blob/master/ml_class_breast_ca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predict Class of Breast Cancer**

data:"breast_cancer.csv"

Data size: approximately 683 individuals with different classes of ovarian cancer :

**Class** : severity

**age** 

**menopause** 

**tumor-size** 

**inv-nodes**

**node-caps**
       
**deg-malig** 

**breast**, 

**breast-quad** 

**irradiat**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import data and preview

In [8]:
## Load modules and libraries
import numpy as np 
import pandas as pd 

## Datapath
datapath = '/content/drive/MyDrive/heamatology_fellows/breast_cancer.csv'

## Load data
df = pd.read_table(datapath, sep=',', header=(0)) 

## Display first few lines of the data
df.head()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,0,5,1,1,1,2,1,3,1,1
1,0,5,4,4,5,7,10,3,2,1
2,0,3,1,1,1,2,2,3,1,1
3,0,6,8,8,1,3,4,3,7,1
4,0,4,1,1,3,2,1,3,1,1


#Exploratory data analysis

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683 entries, 0 to 682
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Class        683 non-null    int64
 1   age          683 non-null    int64
 2   menopause    683 non-null    int64
 3   tumor-size   683 non-null    int64
 4   inv-nodes    683 non-null    int64
 5   node-caps    683 non-null    int64
 6   deg-malig    683 non-null    int64
 7   breast       683 non-null    int64
 8   breast-quad  683 non-null    int64
 9   irradiat     683 non-null    int64
dtypes: int64(10)
memory usage: 53.5 KB


## Column Data Types

In [23]:
df.columns

Index(['Class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps',
       'deg-malig', 'breast', 'breast-quad', 'irradiat'],
      dtype='object')

## Make the statistical summary presentable

## Summary of categories target columns (Class)
Cound the number of yes and NO in each of the respective columns

In [17]:
df.shape

(683, 10)

In [18]:
# Count the number of unique Class
df['Class'].value_counts()
#df['HeartDisease'].value_counts().sum()

0    444
1    239
Name: Class, dtype: int64

## Summary of different factors and their association with the class of breast cancer
Two factor and multilevel associations

crosstab function enables making simple to complex counting of categorical columns according to an index and their related columns.


In [25]:
pd.crosstab(df['Class'], df['tumor-size'], margins=True)

tumor-size,1,2,3,4,5,6,7,8,9,10,All
Class,,,,,,,,,,,
0,344,51,30,12,2,2,2,1,0,0,444
1,2,7,23,31,30,27,28,26,7,58,239
All,346,58,53,43,32,29,30,27,7,58,683


In [26]:
pd.crosstab(df['Class'],
            df['tumor-size'], 
            margins=True).style.background_gradient(cmap='Reds')

tumor-size,1,2,3,4,5,6,7,8,9,10,All
Class,,,,,,,,,,,
0,344,51,30,12,2,2,2,1,0,0,444
1,2,7,23,31,30,27,28,26,7,58,239
All,346,58,53,43,32,29,30,27,7,58,683


### Normalize by row
How many of the Heart Disease patients have a smoking history? 

In [27]:
pd.crosstab(index=df['Class'],
            columns=df['menopause'], 
            margins=True, normalize = 'index')

menopause,1,2,3,4,5,6,7,8,9,10
Class,,,,,,,,,,
0,0.831081,0.083333,0.060811,0.018018,0.000000,0.000000,0.002252,0.002252,0.002252,0.000000
1,0.016736,0.033473,0.104603,0.125523,0.125523,0.104603,0.075314,0.112971,0.020921,0.280335
All,0.546120,0.065886,0.076135,0.055637,0.043924,0.036603,0.027818,0.040996,0.008785,0.098097


In [28]:
pd.crosstab(index=df['Class'],
            columns=df['menopause'], 
            margins = True, normalize = 'index').style.format("{:.0%}").background_gradient(cmap='Reds', axis='index')

menopause,1,2,3,4,5,6,7,8,9,10
Class,,,,,,,,,,
0,83%,8%,6%,2%,0%,0%,0%,0%,0%,0%
1,2%,3%,10%,13%,13%,10%,8%,11%,2%,28%
All,55%,7%,8%,6%,4%,4%,3%,4%,1%,10%


#Exploratory data analysis: Visualization

In order to visulaize, convert the yes/no fields to numerical values by counting them.

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
features = df.drop('Class', axis=1)
labels = df['Class']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [35]:
print(len(labels), len(y_train), len(y_val), len(y_test))

683 409 137 137


In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [37]:
rf = RandomForestClassifier()

scores = cross_val_score(rf, X_train, y_train.values.ravel(), cv=5)

In [39]:
scores

array([0.96341463, 0.93902439, 0.97560976, 0.98780488, 0.95061728])

In [41]:
from sklearn.model_selection import GridSearchCV

In [40]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [42]:
rf = RandomForestClassifier()
parameters = {
    'n_estimators': [5, 50, 100],
    'max_depth': [2, 10, 20, None]
}

cv = GridSearchCV(rf, parameters, cv=5)
cv.fit(X_train, y_train.values.ravel())

print_results(cv)

BEST PARAMS: {'max_depth': None, 'n_estimators': 5}

0.958 (+/-0.02) for {'max_depth': 2, 'n_estimators': 5}
0.958 (+/-0.02) for {'max_depth': 2, 'n_estimators': 50}
0.958 (+/-0.025) for {'max_depth': 2, 'n_estimators': 100}
0.946 (+/-0.037) for {'max_depth': 10, 'n_estimators': 5}
0.961 (+/-0.033) for {'max_depth': 10, 'n_estimators': 50}
0.963 (+/-0.027) for {'max_depth': 10, 'n_estimators': 100}
0.954 (+/-0.019) for {'max_depth': 20, 'n_estimators': 5}
0.958 (+/-0.033) for {'max_depth': 20, 'n_estimators': 50}
0.958 (+/-0.04) for {'max_depth': 20, 'n_estimators': 100}
0.963 (+/-0.027) for {'max_depth': None, 'n_estimators': 5}
0.963 (+/-0.035) for {'max_depth': None, 'n_estimators': 50}
0.961 (+/-0.029) for {'max_depth': None, 'n_estimators': 100}


In [43]:
rf1 = RandomForestClassifier(n_estimators=100, max_depth=10)
rf1.fit(X_train, y_train.values.ravel())

rf2 = RandomForestClassifier(n_estimators=5, max_depth=None)
rf2.fit(X_train, y_train.values.ravel())

rf3 = RandomForestClassifier(n_estimators=50, max_depth=None)
rf3.fit(X_train, y_train.values.ravel())

RandomForestClassifier(n_estimators=50)

In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [46]:
for mdl in [rf1, rf2, rf3]:
    y_pred = mdl.predict(X_val)
    accuracy = round(accuracy_score(y_val, y_pred), 3)
    precision = round(precision_score(y_val, y_pred), 3)
    recall = round(recall_score(y_val, y_pred), 3)
    print('MAX DEPTH: {} / # OF EST: {} -- A: {} / P: {} / R: {}'.format(mdl.max_depth,
                                                                         mdl.n_estimators,
                                                                         accuracy,
                                                                         precision,
                                                                         recall))

MAX DEPTH: 10 / # OF EST: 100 -- A: 0.964 / P: 0.961 / R: 0.942
MAX DEPTH: None / # OF EST: 5 -- A: 0.949 / P: 0.959 / R: 0.904
MAX DEPTH: None / # OF EST: 50 -- A: 0.964 / P: 0.961 / R: 0.942


In [49]:
y_pred = rf1.predict(X_test)
accuracy = round(accuracy_score(y_test, y_pred), 3)
precision = round(precision_score(y_test, y_pred), 3)
recall = round(recall_score(y_test, y_pred), 3)
print('MAX DEPTH: {} / # OF EST: {} -- A: {} / P: {} / R: {}'.format(rf1.max_depth,
                                                                     rf1.n_estimators,
                                                                     accuracy,
                                                                     precision,
                                                                     recall))

MAX DEPTH: 10 / # OF EST: 100 -- A: 0.964 / P: 0.978 / R: 0.917
